In [1]:
import pandas as pd
import numpy as np

In [2]:
def h_prima(H: int, mensaje: str) -> str:
    
    # Se divide el h0 en 4 partes de la siguiente forma
    mascara = 0xFFFFFFFF
    a0 = (H & (mascara << 96)) >> 96
    b0 = (H & (mascara << 64)) >> 64
    c0 = (H & (mascara << 32)) >> 32
    d0 = H & mascara
    
    # Se especifica los shifts por ronda
    s = []
    s[0:15] =  [ 7, 12, 17, 22,  7, 12, 17, 22,  7, 12, 17, 22,  7, 12, 17, 22 ]
    s[16:31] = [ 5,  9, 14, 20,  5,  9, 14, 20,  5,  9, 14, 20,  5,  9, 14, 20 ]
    s[32:47] = [ 4, 11, 16, 23,  4, 11, 16, 23,  4, 11, 16, 23,  4, 11, 16, 23 ]
    s[48:63] = [ 6, 10, 15, 21,  6, 10, 15, 21,  6, 10, 15, 21,  6, 10, 15, 21 ]
      
    k = []
    for i in range(64):
        k.append(int(np.floor(2**32*abs(np.sin(i + 1)))) & mascara)
    
    # Se inicializan las variables
    A = a0
    B = b0
    C = c0
    D = d0
    
    # Se divide el mensaje en chunks de 32 bits
    M = []
    for g in range(0,len(mensaje),4):
        M.append(int.from_bytes(mensaje[g:g+4], byteorder='little'))
    
    for j in range(64):
        F,g = 0,0
        if 0 <= j <= 15:
            F = (B & C) | ((~ B) & D)
            g = j
        elif 16 <= j <= 31:
            F = (D & B) | ((~ D) & C)
            g = (5*j + 1) % 16
        elif 32 <= j <= 47:
            F = B ^ C ^ D
            g = (3*j + 5) % 16
        elif 48 <= j <= 63:
            F = C ^ (B | (~ D))
            g = (7*j) % 16
        
        F = (F + A + k[j] + M[g]) & mascara
        A = D
        D = C
        C = B
        B = (B + (F << s[j] | F >> (32-s[j])) & mascara) & mascara
        
    a0 = (a0 + A) & mascara
    b0 = (b0 + B) & mascara
    c0 = (c0 + C) & mascara
    d0 = (d0 + D) & mascara

    return a0 + (b0 << 32) + (c0 << 64) + (d0 << 96)

In [3]:
def md5_to_hex(digest):
    raw = digest.to_bytes(16, byteorder='little')
    return '{:032x}'.format(int.from_bytes(raw, byteorder='big'))

In [4]:
def custom_md5(m: str, h0: int) -> str:
    # Argumentos:
    #  m: str - mensaje
    # h0: int - constante inicial H_0
    # Retorna:
    #  str - hash MD5 correcto del mensaje en formato hexadecimal
    
    # lo primero es dejar el mensaje divisible por 512
    
    # Ahora se agregarán un uno y ceros hasta que quede divisible por 512
        
    # Se agrega el largo original del mensaje
    message = bytearray(m, 'utf-8') #copy our input into a mutable buffer
    orig_len_in_bits = (8 * len(message)) & 0xffffffffffffffff
    message.append(0x80)
    while len(message)%64 != 56:
        message.append(0)
    message += orig_len_in_bits.to_bytes(8, byteorder='little')

        
    # Ahora se toman los estados para calcular el h_prima
    H = h0
    for i in range(0,len(message),64):
        H = h_prima(H, message[i: i + 64])
        
    return md5_to_hex(H)   

In [5]:
df = pd.read_csv("mensajes_p3.csv", header=None, names=["indice", "mensajes"])
count = 0
mensajes = []
while custom_md5("fcjimenez@uc.cl", 16207084 * 100 + count ) in df["indice"].unique():
    mensajes.append(df[df["indice"] == custom_md5("fcjimenez@uc.cl", 16207084 * 100 + count )]["mensajes"].item())
    count+=1

In [6]:
len(mensajes)

200

In [7]:
def binary_to_text(binary):
    texto = ""
    numeros = []
    for i in range(0, len(binary), 8):
        caracter = int(binary[i: i+8],2)
        numeros.append(caracter)
        caracter = chr(caracter)
        texto += caracter
    return texto, numeros

In [8]:
def propio_xor(m1, m2):
    res = ""
    for i in range(len(m1)):
        if m1[i] == m2[i]:
            res += "0"
        else:
            res += "1"
    return res

In [9]:
def probable_space_count_vector(cyphertext, mensajes):
    length = int(len(cyphertext)/8)
    counts= [0]*length
    
    for c in mensajes:
        messages_xor = propio_xor(c, cyphertext)
        _, numeros = binary_to_text(messages_xor)
        for i in range(len(numeros)):
            if numeros[i] > 64:
                counts[i] += 1
    return [round(c/ len(mensajes), 4) for c in counts]

In [10]:
def max_index(i, lista):
    indice = 0
    max_value = 0
    for j in range(len(lista)):
        if lista[j][i] > max_value:
            indice = j
            max_value = lista[j][i]
    return indice

In [132]:
def pertenecer_grupo(msj, grupo):
    cuantos = 0
    num_por_msj = len(msj)/8
    for integrante in grupo:
        res = propio_xor(msj, integrante)
        _, nums = binary_to_text(res)
        for num in nums:
            if 0 <= num <= 31 or 65 <= num <= 90 or 97 <= num <= 121:
                cuantos += 1
    return cuantos / (len(grupo)*num_por_msj)

In [133]:
def chosen(mensaje, idx):
    contador = 0
    for i in range(0,len(mensaje),8):
        if contador == idx:
            return mensaje[i:i+8]
        contador += 1

In [135]:
def break_random_otp(encrypted_messages: list) -> list :
    # Argumentos:
    #  encrypted_messages: list[str] - lista de mensajes encriptados.
    # Retorna:
    # list[str] - lista de mensajes decriptados
    
    # Creo un diccionario con llave el mensaje y value un booleano que indica si está en un grupo de mensajes o no
    agrupado = dict()
    resultados = dict()
    for m in encrypted_messages:
        agrupado[m] = False
#         resultados[m] = ""
    # Luego reviso todos los mensajes y asigno grupo a los que pueda siempre y cuando no exceda los 15 mensajes
    # por grupo y que la cantidad de xor entre el mensaje que se recorre y el que se compara tenga la misma cantidad que
    # el largo del mensaje, es decir, que el xor que revisamos que un espacio con letras minusculas esté en el rango
    # en los 10 caracteres
    elegidos = []
    for m in encrypted_messages:
        grupo = []
        pase = False
        for m2 in encrypted_messages:
            if m2 != m and not agrupado[m2] and not agrupado[m]:
                resultado = propio_xor(m, m2)
                text, numbers = binary_to_text(resultado)
                cuantos = 0
                for num in numbers:
                    if 0 <= num <= 31 or 65 <= num <= 90 or 97 <= num <= 121:
                        cuantos += 1
                if cuantos == (len(m)/8) and len(grupo) <= 13:
                    grupo.append(m2)
                    pase = True
                    agrupado[m2] = True
        if pase:
            agrupado[m] = True
            grupo.append(m)
            elegidos.append(grupo)
    
    # Luego a cada mensaje que no se le ha asignado un grupo, se le asigna uno
#     print("elegidos: ", elegidos)
    for llave in agrupado:
        if not agrupado[llave]:
            chosen_one = 0
            indices_values = dict()
            for j in range(len(elegidos)):
                indices_values[j] = pertenecer_grupo(llave, elegidos[j])
            indices = sorted(indices_values.items(), key=lambda item: item[1], reverse=True)
#             print("indices: ", indices)
            if len(list(filter(lambda x: len(x) < 15, elegidos))) > 0:
                for tupla in indices:
                    if len(elegidos[tupla[0]]) < 15:
                        elegidos[tupla[0]].append(llave)
                        agrupado[llave] = True
                        break
            else:
                elegidos[indices[0][0]].append(llave)
                agrupado[llave] = True
                
    # Revisa que no haya grupos con menos de 15 mensajes, en caso que haya los reasigna de acuerdo a la probabilidad
    # de que pertenezcan allí. Esta probabilidad se calcula como que corresponda al xor entre una letra minúscula y/o un espacio
    # entre todos los mensajes para un mensaje, es decir, casos favorables de xor con el total de mensaje de dicho grupo
    while len(list(filter(lambda x: len(x) < 15, elegidos))) > 0:
        elegidos.sort(key = len)
        minimo = elegidos.pop(0)
        for msj in minimo:
            indices_values = dict()
            for j in range(len(elegidos)):
                indices_values[j] = pertenecer_grupo(msj, elegidos[j])
            indices = sorted(indices_values.items(), key=lambda item: item[1], reverse=True)
#             print("indices: ", indices)
            if len(list(filter(lambda x: len(x) < 15, elegidos))) > 0:
                for tupla in indices:
                    if len(elegidos[tupla[0]]) < 15:
                        elegidos[tupla[0]].append(msj)
                        break
            else:
                elegidos[indices[0][0]].append(msj)
    
    # Luego se recorre cada grupo y se calcula la probabilidad de que haya un espacio, se crea la posible llave a partir 
    # de este resultado y se realiza el xor para decriptar cada mensaje
#     print(resultados)
    for grupo in elegidos:
        listeilor = []
        res = []
        for m in grupo:
            a = probable_space_count_vector(m, grupo)
            listeilor.append(a)
        max_indices = [max_index(i, listeilor) for i in range(len(listeilor[0]))]
        encrypted_spaces = ""
        for i in range(len(max_indices)):
            encrypted_spaces += chosen(grupo[max_indices[i]],i)
        space = "00100000"*int(len(grupo[0])/8)
        probable_key = propio_xor(encrypted_spaces,space)
        for m in grupo:
            dec = propio_xor(m, probable_key)
            text, _ = binary_to_text(dec)
#             resultados[m] = text
            res.append(text)
        resultados[probable_key] = res
    return resultados

In [136]:
a = break_random_otp(mensajes)

In [137]:
len(a)

13

In [138]:
a

{'01010101011010010101000100000101000101010010101001111100010110010011110101111110': [' {stiYejrR',
  "'aslhR   _",
  '=;sRdQ abX',
  'E ENq a#f7',
  '(c6 uT ,pO',
  'lB \r So;\x13 ',
  'ia<er\x12e[e\x1d',
  '\x0co>k/\x06+Llw',
  'AI7j8\x10&L[p',
  "C]vf8\x11'\nfv",
  '\x12.O\x02vhke53',
  '/\x00>\x1aX\x1dXv\x15:',
  'hA\x13<c7pA\x08R',
  'W BUpiz jc',
  "['@Ags e\x12,"],
 '00100100001010100010101101001000010011100100111101000010001010010101000101000110': ['v^w    H o',
  'vPvte ?A, ',
  ' Zq \x12(=^x ',
  'x ~~\x10%  \x037',
  '\x1bG hoRr>ul',
  'vQlte5!Te!',
  "vqlwe2'Lb+",
  '<#w\r^FhRep',
  'dkE\x16\x10SsHyp',
  "ewH\x14YP\x7fX'p",
  '\x1b\x19\x11\x13Y*P\x0e~X',
  '\x16U#N\x1bxl\x0ecZ',
  '^\x0c\x1b\x15\x1bxw\x0e~\x12',
  'L\x16ik%GE)p"',
  'HmbtnCL%\x12b'],
 '01000001001011000011011101010011011110000110010000101010010110110111100001101110': ['  fu OYV F',
  "$cpi'CZCdH",
  '6crl=MP_7H',
  's+qrtOIW6H',
  'ey  o  Q*A',
  '=04b;JB\x14e ',
  's\rqv1\\\x1bI-\x07',
  '\'b4At]RT"\x04',
